In [2]:
import sys
import os
import re
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.size'] = 12

In [7]:
dt_outpath = '../data/'
fig_outpath = '../figures/'
bash_outpath = './sh/'

## STAR 2 step alignment

In [11]:
path_to_ref = '/private/groups/brookslab/gabai/tools/ref/yst/star_aresv13/'
path_to_fastq = '/private/groups/brookslab/data.rep/yeast_shortread_rnaseq/'
outpath = '/private/groups/brookslab/gabai/projects/yeastMeth/data/rna/totalrna/rsem/'

In [4]:
fs = os.listdir(path_to_fastq)
fastq_prefix = [re.split(r'_R.*_001\.fastq\.gz', f)[0] for f in fs if f.endswith('fastq.gz')]
fastq_prefix = sorted(set(fastq_prefix))

In [6]:
samples = ['ys18_rep1', 'ys18_rep2', 'ys18_rep3', 'ym209_rep1', 'ym209_rep2', 'ym209_rep3']

In [10]:
for s, fq in zip(samples, fastq_prefix):
    outfile= open(f'{bash_outpath}250804_runSTAR_2step_on_aresv13_{s}.sh', 'w')
    header = f'#!/bin/bash\n#SBATCH --job-name=yeast_totalRNA_STAR_step1_%s\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=32\n#SBATCH --nodes=1\n#SBATCH --time=120\n#SBATCH --mem=50G\n#SBATCH --partition=medium\n#SBATCH --error={outpath}log/lsf_%s.err\n#SBATCH --output={outpath}log/lsf_%s.out\n#SBATCH --mail-type=END,FAIL\n#SBATCH --mail-user=gabai@ucsc.edu\n' % (s, s, s)
    cmd1 = f'/private/groups/brookslab/bin/STAR-2.7.11b/bin/Linux_x86_64_static/STAR --runThreadN 16 --genomeDir {path_to_ref} --readFilesIn {path_to_fastq}{fq}_R1_001.fastq.gz {path_to_fastq}{fq}_R2_001.fastq.gz {precise_params} --outFileNamePrefix {outpath}{s}\n'
    outfile.write(header)
    outfile.write('\n')
    outfile.write(cmd1)
    outfile.close()

## RSEM quantification

In [12]:
path_to_bam = '/private/groups/brookslab/gabai/projects/yeastMeth/data/rna/totalrna/star_2step/'
path_to_rsem_ref = '/private/groups/brookslab/gabai/tools/ref/yst/rsem_ares_v13'
outpath = '/private/groups/brookslab/gabai/projects/yeastMeth/data/rna/totalrna/rsem/'

In [13]:
precise_params = f"""--no-bam-output \\
    --seed 12345 -p 16 \\
    --strandedness reverse \\
    --alignments \\
    --paired-end"""

In [16]:
for s in samples:
    outfile= open(bash_outpath + '250807_totalrna_rsem_quant_%s.sh'%s, 'w')
    header = f'#!/bin/bash\n#SBATCH --job-name=yeast_totalrna_rsem_%s\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=20\n#SBATCH --nodes=1\n#SBATCH --time=120\n#SBATCH --mem=100G\n#SBATCH --partition=medium\n#SBATCH --error={outpath}log/lsf_%s.err\n#SBATCH --output={outpath}log/lsf_%s.out\n#SBATCH --mail-type=END,FAIL\n#SBATCH --mail-user=gabai@ucsc.edu\n' % (s, s, s)
    cmd = f'/private/groups/brookslab/bin/RSEM-1.3.0/rsem-calculate-expression {precise_params} {path_to_bam}{s}Aligned.toTranscriptome.out.pass.sorted.pass.rname.sorted.bam {path_to_rsem_ref} {outpath}{s}'
    outfile.write(header)
    outfile.write('\n')
    outfile.write(cmd)
    outfile.close()